In [1]:
import pandas as pd
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
frame_test = pd.read_csv("hq_markup_train.csv")
frame = pd.read_csv("markup_train.csv")

def get_sample(index, frame):
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data


def find_empty_indexes_2(data):
    """
    Поиск id файлов из строк датафрейма для их исключения на этапе удаления данных из датафрейма
    """
    B=[]
    for i in range(data.shape[0]):
        try:
            df=get_sample(i,data)
        except ValueError:
            B.append(data.iloc[i,0])
    return B


class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe:pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        x = get_sample(idx, self.siam_dataset_describe) #.to_numpy(dtype=np.float64)
        # t = x["time"].to_numpy(dtype=np.float64)
        x = x[["delta_p", "p_"]].to_numpy(dtype=np.float64)
        


        # 7) Возвращаем (X, Y)
        return x, self.siam_dataset_describe.iloc[idx][['Некачественное ГДИС', 'Влияние ствола скважины', 'Радиальный режим', 'Линейный режим', 'Билинейный режим', 'Сферический режим', 'Граница постоянного давления', 'Граница непроницаемый разлом']].to_numpy(dtype=np.int8)

# B=find_empty_indexes_2(frame)#Поиск id с отсутствующими файлами для исключения
# frame=frame[~frame['file_name'].isin(B)]#Фильтрация данных
# frame = frame[~frame["file_name"].isin(frame_test["file_name"])]

In [9]:
def collate_fn_varlen(batch):
    """
    batch: список [(x_i, y_i), (x_j, y_j), ...].
    Выравниваем каждый x_i внутри батча до max_len батча,
    возвращаем (x_padded, lengths, y_tensor).
    """
    max_len = max(x.shape[0] for x, _ in batch)
    batch_size = len(batch)
    input_dim = batch[0][0].shape[1]
    num_classes = batch[0][1].shape[0]
    
    x_padded = torch.zeros(batch_size, max_len, input_dim, dtype=torch.float32)
    lengths = torch.zeros(batch_size, dtype=torch.long)
    y_tensor = torch.zeros(batch_size, num_classes, dtype=torch.float32)
    
    for i, (x_i, y_i) in enumerate(batch):
        T_i = x_i.shape[0]
        x_padded[i, :T_i, :] = torch.from_numpy(x_i)
        lengths[i] = T_i
        y_tensor[i] = torch.from_numpy(y_i)
    
    return x_padded, lengths, y_tensor

class LSTMSingleClassModel(nn.Module):
    """
    LSTM-модель для бинарной классификации (один выход).
    Использует pack_padded_sequence для эффективной обработки переменной длины.
    """
    def __init__(self, input_dim, hidden_dim, bidirectional=False, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )
        dir_factor = 2 if bidirectional else 1
        self.classifier = nn.Linear(hidden_dim * dir_factor, 1)  # один выход!
    
    def forward(self, x_padded, lengths):
        packed_input = nn.utils.rnn.pack_padded_sequence(
            x_padded, 
            lengths.cpu(),
            batch_first=True, 
            enforce_sorted=False
        )
        
        packed_output, (h_n, c_n) = self.lstm(packed_input)
        
        # Выделяем финальное скрытое состояние
        if self.lstm.bidirectional:
            # h_n shape: [num_layers*2, batch_size, hidden_dim]
            # Возьмём последний слой h_n[-2], h_n[-1] и склеим
            num_layers = self.lstm.num_layers
            hidden_dim = self.lstm.hidden_size
            batch_size = x_padded.size(0)
            
            # Развёртываем: [num_layers, 2, batch_size, hidden_dim]
            h_last_layer = h_n.view(num_layers, 2, batch_size, hidden_dim)
            h_forward = h_last_layer[-1, 0, :, :]   # [batch_size, hidden_dim]
            h_backward = h_last_layer[-1, 1, :, :]  # [batch_size, hidden_dim]
            h_final = torch.cat([h_forward, h_backward], dim=1)  # [batch_size, 2*hidden_dim]
        else:
            # h_n[-1] => [batch_size, hidden_dim]
            h_final = h_n[-1]
        
        logits = self.classifier(h_final)  # [batch_size, 1]
        return logits


In [3]:
def train_single_class(
    model, 
    data_loader, 
    device, 
    lr=1e-3, 
    num_epochs=5, 
    class_index=0
):
    """
    Обучает model (LSTMSingleClassModel) предсказывать класс с индексом class_index.
    data_loader: выдаёт (x_padded, lengths, y_full), где y_full имеет shape [batch_size, num_classes].
                 мы берём только y_full[:, class_index].
    """
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for x_padded, lengths, y_full in data_loader:
            x_padded = x_padded.to(device)
            lengths = lengths.to(device)
            
            # Вытаскиваем бинарную метку для нужного класса c:
            y_target = y_full[:, class_index].to(device)  # shape [batch_size]
            
            optimizer.zero_grad()
            logits = model(x_padded, lengths).squeeze(-1)  # shape [batch_size, 1] => [batch_size]
            
            loss = criterion(logits, y_target)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(data_loader)
        print(f"[Class {class_index}] Epoch {epoch+1}/{num_epochs}, Loss={avg_loss:.4f}")
    
    return model


In [18]:
def train_one_vs_all_models():
    # Параметры
    
    input_dim = 2
    num_classes = 8
    hidden_dim = 256
    bidirectional = True
    batch_size = 128
    num_epochs = 10
    num_layers = 2
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Загружаем датасет
    dataset = SiamDataset(frame)
    data_loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        collate_fn=collate_fn_varlen
    )
    
    # Для каждого класса создадим свою модель
    models = []
    for c in range(num_classes):
        print(f"=== Обучаем модель для класса {c} (one-vs-all) ===")
        
        model_c = LSTMSingleClassModel(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            bidirectional=bidirectional,
            num_layers= num_layers
        ).to(device)
        
        # Обучаем
        trained_model_c = train_single_class(
            model_c, 
            data_loader,
            device=device,
            lr=1e-3,
            num_epochs=num_epochs,
            class_index=c
        )
        
        models.append(trained_model_c)
    
    print("Все модели успешно обучены.")
    return models

if __name__ == "__main__":
    all_class_models = train_one_vs_all_models()


=== Обучаем модель для класса 0 (one-vs-all) ===
[Class 0] Epoch 1/10, Loss=0.3478
[Class 0] Epoch 2/10, Loss=0.3244
[Class 0] Epoch 3/10, Loss=0.3108
[Class 0] Epoch 4/10, Loss=0.2978
[Class 0] Epoch 5/10, Loss=0.2839
[Class 0] Epoch 6/10, Loss=0.2805
[Class 0] Epoch 7/10, Loss=0.2750
[Class 0] Epoch 8/10, Loss=0.2723
[Class 0] Epoch 9/10, Loss=0.2677
[Class 0] Epoch 10/10, Loss=0.2867
=== Обучаем модель для класса 1 (one-vs-all) ===
[Class 1] Epoch 1/10, Loss=0.4226
[Class 1] Epoch 2/10, Loss=0.4044
[Class 1] Epoch 3/10, Loss=0.3994
[Class 1] Epoch 4/10, Loss=0.3956
[Class 1] Epoch 5/10, Loss=0.3921
[Class 1] Epoch 6/10, Loss=0.3798
[Class 1] Epoch 7/10, Loss=0.3746
[Class 1] Epoch 8/10, Loss=0.3693
[Class 1] Epoch 9/10, Loss=0.3662
[Class 1] Epoch 10/10, Loss=0.3613
=== Обучаем модель для класса 2 (one-vs-all) ===
[Class 2] Epoch 1/10, Loss=0.5851
[Class 2] Epoch 2/10, Loss=0.5610
[Class 2] Epoch 3/10, Loss=0.5460
[Class 2] Epoch 4/10, Loss=0.5380
[Class 2] Epoch 5/10, Loss=0.5452
[

In [ ]:
def predict_one_vs_all(models, x, device, threshold=0.5):
    """
    models: список, где models[c] - LSTMSingleClassModel для класса c
    x: np.array shape [T, input_dim] (один объект) 
    """
    # Превращаем в батч размером 1
    T = x.shape[0]
    x_tensor = torch.from_numpy(x).unsqueeze(0)  # [1, T, input_dim]
    lengths = torch.tensor([T], dtype=torch.long)
    
    # Будем хранить предсказания [num_classes]
    preds = []
    
    for c, model_c in enumerate(models):
        model_c.eval()
        with torch.no_grad():
            logits = model_c(x_tensor.to(device), lengths.to(device))
            # logits shape: [1, 1]
            prob = torch.sigmoid(logits)[0, 0].item()
            pred_label = 1 if prob >= threshold else 0
            preds.append(pred_label)
    
    return preds  # список [0/1] длиной num_classes


In [11]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, accuracy_score

def evaluate_one_vs_all_models(models, data_loader, threshold=0.5):
    """
    Оценивает набор моделей (one-vs-all) по метрикам precision, recall, f1, hamming_loss.
    
    Параметры:
    ----------
    models : list of nn.Module
        Список моделей, где models[c] - это LSTMSingleClassModel (или любая бинарная модель)
        для класса c.
    data_loader : DataLoader
        Должен возвращать батчи в формате (x_padded, lengths, y_true), где:
          - x_padded: [batch_size, max_len, input_dim]
          - lengths: [batch_size] (длины последовательностей)
          - y_true: [batch_size, num_classes] (истинные метки для всех классов)
    device : torch.device
        'cuda' или 'cpu'
    threshold : float
        Порог для перевода предсказанных вероятностей в 0/1.
    
    Возвращает:
    -----------
    metrics_dict : dict
        Словарь вида {
          'precision_macro': ...,
          'recall_macro': ...,
          'f1_macro': ...,
          'hamming_loss': ...
        }
    """
    # Определим количество классов на основе длины списка моделей
    num_classes = len(models)
    
    # Переведём все модели в режим eval
    for m in models:
        m.eval()
    
    all_preds = []
    all_labels = []
    all_probs = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        for x_padded, lengths, y_true in data_loader:
            # x_padded: [batch_size, max_len, input_dim]
            # lengths: [batch_size]
            # y_true: [batch_size, num_classes]
            x_padded = x_padded.to(device)
            lengths = lengths.to(device)
            
            # Будем собирать предсказания по всем классам для этого батча
            # batch_preds: [batch_size, num_classes]
            batch_preds = []
            batch_probs = []
            for c, model_c in enumerate(models):
                # Прогоняем i-й класс
                logits = model_c(x_padded, lengths).squeeze(-1)  # [batch_size]
                probs = torch.sigmoid(logits)                    # [batch_size]
                preds_c = (probs >= threshold).long()            # [batch_size]
                # print(logits)
                # Расширяем до [batch_size, 1] и добавим в список
                batch_preds.append(preds_c.unsqueeze(-1))
                batch_probs.append(logits.unsqueeze(-1))
            
            # Склеим вдоль последней оси => [batch_size, num_classes]
            # print(batch_probs)
            batch_preds = torch.cat(batch_preds, dim=1)
            batch_probs = torch.cat(batch_probs, dim=1)
            all_probs.append(batch_probs.cpu().numpy())
            all_preds.append(batch_preds.cpu().numpy())
            all_labels.append(y_true.numpy())  # y_true уже float на CPU (смотри коллектор)

    # Склеиваем все батчи в один массив
    all_probs = np.concatenate(all_probs, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)   # [N, num_classes]
    all_labels = np.concatenate(all_labels, axis=0) # [N, num_classes]
    
    # Метрики. Можно менять 'macro'/'micro' и т.д. по вкусу.
    prec_macro = precision_score(all_labels, all_preds, average='micro', zero_division=0)
    rec_macro  = recall_score(all_labels, all_preds, average='micro', zero_division=0)
    f1_macro   = f1_score(all_labels, all_preds, average='micro', zero_division=0)
    accuracy = accuracy_score(all_labels.ravel(), all_preds.ravel())
    h_loss     = hamming_loss(all_labels, all_preds)
    print(all_preds)
    print(all_labels)
    with open("./preds_labels.txt", "w") as f:
        f.write("[" + ",".join([str(i) for prob in all_probs for i in prob]) + "]")
        
    metrics_dict = {
        'precision_macro': prec_macro,
        'recall_macro': rec_macro,
        'f1_macro': f1_macro,
        "accuracy": accuracy,
        'hamming_loss': h_loss
    }
    print(f"Accuracy по полным классам {np.sum(np.all(all_preds == all_labels, axis=1))/all_preds.shape[0]}")
    return metrics_dict

dataset = SiamDataset(frame_test)
data_loader = DataLoader(
        dataset, 
        batch_size=16, 
        shuffle=False, 
        collate_fn=collate_fn_varlen
    )

evaluate_one_vs_all_models(loaded_models, data_loader, threshold=0.4)

[[0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 1]
 [0 1 1 ... 0 1 0]
 [0 1 1 ... 0 0 0]]
[[0. 1. 1. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 1. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
Accuracy по полным классам 0.26


{'precision_macro': 0.7452076677316294,
 'recall_macro': 0.7566909975669099,
 'f1_macro': 0.7509054325955734,
 'accuracy': 0.84525,
 'hamming_loss': 0.15475}

In [ ]:
import torch
import os

# Папка для сохранения моделей
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Сохраняем каждую модель
for idx, model in enumerate(all_class_models):
    # Сохраняем состояние модели
    model_path = os.path.join(save_dir, f"modelv5_withoutGood_class_{idx}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"Модель для класса {idx} сохранена в {model_path}")

Модель для класса 0 сохранена в saved_models\modelv4_withoutGood_class_0.pt
Модель для класса 1 сохранена в saved_models\modelv4_withoutGood_class_1.pt
Модель для класса 2 сохранена в saved_models\modelv4_withoutGood_class_2.pt
Модель для класса 3 сохранена в saved_models\modelv4_withoutGood_class_3.pt
Модель для класса 4 сохранена в saved_models\modelv4_withoutGood_class_4.pt
Модель для класса 5 сохранена в saved_models\modelv4_withoutGood_class_5.pt
Модель для класса 6 сохранена в saved_models\modelv4_withoutGood_class_6.pt
Модель для класса 7 сохранена в saved_models\modelv4_withoutGood_class_7.pt


In [10]:
loaded_models = []
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = 2
num_classes = 8
hidden_dim = 256
bidirectional = True
batch_size = 128
num_epochs = 10
num_layers = 2
save_dir = "saved_models"

for idx in range(num_classes):
    # Создаём экземпляр модели
    model_c = LSTMSingleClassModel(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        bidirectional=bidirectional,
        num_layers=num_layers
    ).to(device)
    
    # Загружаем сохранённые веса
    model_path = os.path.join(save_dir, f"modelv4_withoutGood_class_{idx}.pt")
    model_c.load_state_dict(torch.load(model_path))
    model_c.eval()  # Переводим модель в режим оценки
    
    loaded_models.append(model_c)
    print(f"Модель для класса {idx} загружена из {model_path}")

Модель для класса 0 загружена из saved_models\modelv4_withoutGood_class_0.pt
Модель для класса 1 загружена из saved_models\modelv4_withoutGood_class_1.pt
Модель для класса 2 загружена из saved_models\modelv4_withoutGood_class_2.pt
Модель для класса 3 загружена из saved_models\modelv4_withoutGood_class_3.pt
Модель для класса 4 загружена из saved_models\modelv4_withoutGood_class_4.pt
Модель для класса 5 загружена из saved_models\modelv4_withoutGood_class_5.pt
Модель для класса 6 загружена из saved_models\modelv4_withoutGood_class_6.pt
Модель для класса 7 загружена из saved_models\modelv4_withoutGood_class_7.pt
